In [10]:
import pandas as pd
import gymnasium as Env

In [23]:
FX_data = pd.read_parquet("../data/FX_data.parquet.gzip")

In [25]:
FX_data

ccy,eurjpy,eurusd,sgdjpy,usdjpy,usdsgd
timestamp,,,,,
2010-01-04 09:00:00,133.650,1.44300,66.220,92.630,1.39830
2010-01-04 09:01:00,133.640,1.44280,66.230,92.640,1.39830
2010-01-04 09:02:00,133.650,1.44270,66.240,92.640,1.39820
2010-01-04 09:03:00,133.670,1.44310,66.240,92.640,1.39820
2010-01-04 09:04:00,133.710,1.44390,66.220,92.620,1.39810
...,...,...,...,...,...
2024-12-31 16:54:00,162.825,1.03546,115.108,157.224,1.36508
2024-12-31 16:55:00,162.831,1.03554,115.057,157.219,1.36507
2024-12-31 16:56:00,162.801,1.03539,115.040,157.211,1.36508


In [34]:
def create_reverse_tickers(historical_data: pd.DataFrame) -> pd.DataFrame:
    """
    If USDEUR is present, create EURUSD as 1 / USDEUR
    """
    initial_columns = historical_data.columns
    for fx in initial_columns:
        reverse_rate = fx[-3:] + fx[:3]
        if reverse_rate not in initial_columns:
            historical_data[reverse_rate] = 1 / historical_data[fx]
    return historical_data

In [35]:
def get_unique_currencies(historical_data: pd.DataFrame) -> set:
    return {y for x in FX_data.columns for y in (x[:3], x[-3:])}

In [122]:
class FxTradingEnv(gym.Env):

    def __init__(
        self,
        historical_prices: pd.DataFrame,
        initial_portfolio: dict[str, float],
        long_position_fee: float = 0.1,
        long_only: bool = True,
        short_position_fee: float = 0.1,  # todo: add shorting later
        base_currency: str = "usd",
        start_datetime: pd.Timestamp = None,
    ):
        """
        Gymnasium environment for FX trading

        Assumtions:
            1. We cannot make multiple-currency trades at the same time
            For example if we have USD and we want to buy SGDJPY, then we have to buy USDSGD or USDJPY first
        """
        self.current_portfolio = initial_portfolio
        self.historical_prices = historical_prices
        self.base_currency = base_currency
        self.fees = {"long": long_position_fee, "short": short_position_fee}
        self.long_only = long_only
        self.datetime = start_datetime or historical_prices.index.min()

    def preprocess_data(self) -> None:
        """
        1. Validate inputs
        2. Create reverse tickers
        """
        self._validate_inputs()
        self.historical_prices = create_reverse_tickers(self.historical_prices)

    def _validate_inputs(self) -> None:
        """
        Check validity of price history and current portfolio
        """

        self.all_currencies = get_unique_currencies(self.historical_prices)

        for x in self.all_currencies:
            if x not in self.current_portfolio:
                print(f"{x} not in inital portfolio. Setting quantity to 0")
                self.current_portfolio[x] = 0.0

        for x in self.current_portfolio:
            if x not in self.all_currencies:
                raise KeyError(f"ccy {x} has no history")

        if self.datetime not in self.historical_prices.index:
            raise KeyError(f"{self.datetime} is missing in data")

    def _convert_portfolio_to_base_ccy(self) -> dict:
        """
        converts portfolio to base currency
        """
        portfolio_in_base_ccy = {}

        for ccy_name, amount in self.current_portfolio.items():

            if ccy_name == self.base_currency:
                mult = 1.0
            else:
                mult = float(current_market[self.base_currency + ccy_name])

            portfolio_in_base_ccy[ccy_name] = amount / mult

        return portfolio_in_base_ccy

    @property
    def current_portfolio_value(self):
        """
        Get current portfolio value in base currency
        """
        return sum(self._convert_portfolio_to_base_ccy().values())

    @property
    def current_portfolio_weights(self):
        """
        Get current portfolio weights
        """
        portfolio = self._convert_portfolio_to_base_ccy()
        total_value = sum(portfolio.values())
        return {ccy: value / total_value for ccy, value in portfolio.items()}

Todo:

Create ABC

Tests:

1. Unknown currency in initial portfolio
2. Missing currency in initial portfolio
3. Portfolio value in base
4. Portfolio weights == 1

In [123]:
current_porfolio = {
    "usd": 100_000,
    "eur": 100_000,
    # "jpy": 100_000,
    "sgd": 100_000,
}

In [124]:
fx_env = FxTradingEnv(
    initial_portfolio=current_porfolio,
    historical_prices=FX_data,
)

In [125]:
fx_env.preprocess_data()

jpy not in inital portfolio. Setting quantity to 0


In [126]:
fx_env.current_portfolio

{'usd': 100000, 'eur': 100000, 'sgd': 100000, 'jpy': 0.0}

In [127]:
fx_env._convert_portfolio_to_base_ccy()

{'usd': 100000.0, 'eur': 144300.0, 'sgd': 71515.41157119359, 'jpy': 0.0}

In [128]:
fx_env.current_portfolio_value

315815.4115711936

In [129]:
fx_env.current_portfolio_weights

{'usd': 0.3166406588661909,
 'eur': 0.4569124707439135,
 'sgd': 0.22644687038989553,
 'jpy': 0.0}